# Data Wrangling & Analysis Take Home Project: Cleaning FERC Form 1

---------

## This is not an exam.
* This task is an example of the kind of work we do to make public energy data usable for analysis.
* We don’t expect you to come up with a clean, comprehensive solution.
* We want to be able to explore your general approach together and see how you think about these kinds of problems.
* Spend 2-4 hours working on it. This doesn't have to happen all at once. We want you to have time to play with the data, step away from it to think, and then come back to it again.
* Feel free to use whatever documentation or online resources you would normally consult while working on a data wrangling problem.
* Feel free to use additional 3rd party libraries if you want to.  You should be able to install them from within the notebook using `!pip install packagename` or `!conda install packagename`

## Email us your notebook within a week.
* Send it to [hello@catalyst.coop](mailto:hello@catalyst.coop) (normally we'd have you make a PR but... we don't want everyone looking at each others solutions)
* We'll review your notebook and if it looks good, we'll reach out to schedule a longer conversation about it, and another technical interview.

## Some questions to keep in mind:
* What assumptions are you making about the data?
* How will you test whether / when those assumptions are valid?
* How would you / did you deal with the data that don’t conform to those assumptions?
* If there are records which can’t be reasonably cleaned automatically, but were high value in an advocacy context, how would you integrate manual cleaning into the automated process so that the manual effort is captured, and can be incrementally improved over time?
* What expectations do you have about the output data?
* What kind of data validation checks would you design to make sure that the output meets your expectations? These could be either integrated into the table transformation process, or run on the final output.
* How do you decide when data isn’t recoverable?
* How will you evaluate the completeness of the data that you’ve been able to extract?
* What kind of queries are you trying to make easy with the structure of the output data?
* What parts of this process might make sense to generalize / abstract for re-use in extracting, cleaning, and reorganizing data from other tables?

# Background on the FERC Form 1 Database
* The FERC Form 1 collects financial data about electric utilities in the US. It’s a treasure-trove of information if you want to understand how these utilities make and spend money. The capital they have locked up in existing fossil fuel infrastructure is one of the big reasons they fight against the transition to clean energy. Data from the FERC Form 1 can help advocates understand which utilities will be easiest to engage in the transition, and which ones may be hopeless pyromaniacs.
* Unfortunately, FERC does not organize its data very well, or do much quality control, so this data is difficult to extract and use. We’ve built a script that pulls together all of FERC’s annual Visual FoxPro databases into a single SQLite database covering all the years of data. Then we write extract and transform functions to pull tables from this multi-year DB and clean them up for easier analysis.
* To help us understand how you approach working with messy data and turning it into something usable, we’d like you to develop a strategy for reshaping and cleaning the data in a couple of these tables.
* [Here is some documentation about the FERC Form 1 Database](https://catalystcoop-pudl.readthedocs.io/en/dev/data_sources/ferc1_db_notes.html), including a mapping between database tables and the pages of the PDF that their data is collected from.

## Other things you should know about the FERC database structure:
  * The FERC Form 1 database generally mirrors the structure of the old paper forms that were used to collect this data.
  * Columns on the paper form have been translated into columns in the database, and the numbered rows on the paper forms are identified by a `row_number` field in the database.
  * Each `row_number` is associated with a different reported value, meaning that different records in the FERC 1 database can represent observations of many different variables.
  * In addition, what `row_number` corresponds to what variable has changed over the years as new rows have been added to the form and old ones have been removed or split into multiple rows offering more granular information.
  * The `f1_row_lit_tbl` maps combinations of table names, row numbers, and years to the description that applied to that row number, and thus indicates how the meanings associated with individual rows have changed over time. The `row_chg_yr` column indicates the last year in which the meaning of a row number changed.

# Set up access to the FERC Form 1 DB
* You can download a copy of our FERC Form 1 SQLite DB from: https://data.catalyst.coop/ferc1.db
* Substitute the path to that file on your system below:

In [1]:
import sqlalchemy as sa
import pandas as pd

FERC1_DB_PATH = "/path/to/your/copy/of/ferc1.db"
ferc1_engine = sa.create_engine(f"sqlite:///{FERC1_DB_PATH}")

# **Task**: Data Cleaning
Examine the Small Plants table: `f1_gnrt_plant` and the [blank Form 1 (PDF)](https://catalystcoop-pudl.readthedocs.io/en/dev/_downloads/6a316a949a522f595e7575b6fd7034b8/ferc1_blank_2022-11-30.pdf) describing the information being collected in this table.
 This table describes smaller power plants in the US, including their date of construction, fuel types, fuel consumption, net electricity generation, operational and maintenance costs. Unfortunately, the table does not provide a unique ID for each power plant so it’s difficult to assemble a complete time series tracking the plant across all the years of available data. In general the table also doesn’t use controlled vocabularies for variables like the type of plant or fuel.

**Design a process for cleaning up the data in this table so that it can be used for analysis, including tracking individual plants across years, and ensuring that the values reported within each column are comparable. Derived values of interest might be the total per-plant cost per Megawatt-hour of electricity generated each year, or the per-plant quantity of fuel consumed per Megawatt-hour of electricity generated each year.** 

In some cases, information that should be part of a single record is spread across multiple rows. Sometimes numeric IDs and names are stored in the same field. It also seems like different records may be reporting the same kind of value in a given column, but using different units of measurement (e.g. kilowatts vs. megawatts, or tons of coal vs. gallons of diesel fuel). You can ignore the columns whose names end with `_f` as they refer to footnotes.

In [6]:
small_plants = pd.read_sql("f1_gnrt_plant", ferc1_engine)
small_plants.sample(10)

,respondent_id,report_year,spplmnt_num,row_number,row_seq,row_prvlg,plant_name,yr_constructed,capacity_rating,net_demand,...,net_demand_f,net_generation_f,plant_cost_f,plant_cost_mw_f,operation_f,expns_fuel_f,expns_maint_f,kind_of_fuel_f,fuel_cost_f,report_prd
1941,34,1996,0,17,0,N,provided.,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,12
17669,3,2017,0,15,15,,The size of the penstock limits the maximum,,0.00,0.0,...,0,0,0,0,0,0,0,0,0,12
9679,73,2006,0,5,5,,Elkhart (Project #2651),1913,3.44,4.0,...,0,0,0,0,0,0,0,0,0,12
7510,41,2002,0,8,8,N,Croton - FPC #2468,1907,8.85,8.1,...,0,0,0,0,0,0,0,0,0,12
15731,120,2015,0,10,10,,West Faribault,1965,0.00,0.0,...,0,0,0,0,0,0,0,0,0,12
8614,11,2004,0,6,6,,Bar Harbor,,8000.00,0.0,...,0,0,0,0,0,0,0,0,0,12
9710,294,2006,0,4,4,,INTERNAL COMBUSTION:,,0.00,0.0,...,0,0,0,0,0,0,0,0,0,12
11790,151,2009,0,11,11,,Notes:,,0.00,0.0,...,0,0,0,0,0,0,0,0,0,12
8699,161,2004,0,35,35,,Bishop Creek No. 6,1913,1.60,0.0,...,0,0,0,0,0,0,0,0,0,12
7309,29,2002,0,7,7,N,,1968,0.00,0.0,...,0,0,0,0,0,0,0,0,0,12


In [7]:
small_plants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19559 entries, 0 to 19558
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   respondent_id      19559 non-null  int64  
 1   report_year        19559 non-null  int64  
 2   spplmnt_num        19559 non-null  int64  
 3   row_number         19559 non-null  int64  
 4   row_seq            19559 non-null  int64  
 5   row_prvlg          19559 non-null  object 
 6   plant_name         19559 non-null  object 
 7   yr_constructed     19559 non-null  object 
 8   capacity_rating    18909 non-null  float64
 9   net_demand         18663 non-null  float64
 10  net_generation     11936 non-null  float64
 11  plant_cost         13479 non-null  float64
 12  plant_cost_mw      12277 non-null  float64
 13  operation          10736 non-null  float64
 14  expns_fuel         3258 non-null   float64
 15  expns_maint        11553 non-null  float64
 16  kind_of_fuel       195